In [ ]:
import pandas as pd
import numpy as np
from ggv.utils.context import create_default_context
import warnings
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly
from plotly.subplots import make_subplots
import datetime
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
context = create_default_context()
bq_db= context.get_ds('YOUR CONTEXT')

In [ ]:
query= f""" 
WITH
  distinct_loc AS (
  SELECT
    DISTINCT driver_id,
    DATETIME(location_updated_at, 'Asia/Hong_Kong')AS location_updated_at,
    location
  FROM
    `TableName`
  WHERE
    DATE(location_updated_at) BETWEEN DATE('2023-05-31')
    AND DATE('2023-05-31') ),
  driver_loc AS (
  SELECT
    driver_id,
    ST_GEOGFROMGEOJSON(raw.parse_wkx(JSON_VALUE(location, "$.wkb"))) AS location,
    location_updated_at
  FROM
    distinct_loc),
  driver_locat AS (
    SELECT
      driver_id,
      ST_X(location) AS driver_lon,
      ST_Y(location) AS driver_lat,
      location_updated_at
    FROM driver_loc
  ),
  accepted_orders AS (
  SELECT
    do.courier_id,
    do.id,
    do.created_at,
    do.source_ts,
    o.pickup_location_lat,
    o.pickup_location_lon,
    o.destination_location_lat,
    o.destination_location_lon
  FROM
    `TableName` DO
  LEFT JOIN
    `TableName` o
  ON
    do.id = o.system_order_request_id
  WHERE
    DATE(do.source_ts) BETWEEN DATE('2023-05-31')
    AND DATE('2023-05-31')
    AND do.status = 'accepted'
    AND product_name = 'DELIVERY'),
  hand AS (
  SELECT
    DISTINCT courier_id,
    id AS order_id,
    MAX(DATETIME(source_ts,'Asia/Hong_Kong')) AS event_time
  FROM
    `TableName`
  WHERE
    status IN('accepted',
      'picked_up')
  GROUP BY
    1,
    2),
  completed AS (
  SELECT
    courier_id,
    id AS order_id,
    MIN(DATETIME(source_ts,'Asia/Hong_Kong')) AS event_time
  FROM
    `TableName`
  WHERE
    status = 'completed'
  GROUP BY
    1,
    2 ),
  last_location AS (
  SELECT
    id AS order_id,
    courier_id,
    MAX(location_updated_at) AS last_location_updated_at
  FROM
    accepted_orders
  LEFT JOIN
    driver_loc
  ON
    driver_loc.driver_id = accepted_orders.courier_id
  WHERE
    driver_loc.location_updated_at <= DATETIME(accepted_orders.source_ts,'Asia/Hong_Kong')
  GROUP BY
    1,
    2 )
SELECT
  accepted_orders.id AS order_id_accepted,
  accepted_orders.courier_id,
  DATETIME(source_ts,'Asia/Hong_Kong') AS accepted_time,
  accepted_orders.created_at,
  hand.order_id AS order_id_in_hand,
  location_updated_at,
  accepted_orders.pickup_location_lat AS accepted_order_pickup_lat,
  accepted_orders.pickup_location_lon AS accepted_order_pickup_lon,
  accepted_orders.destination_location_lat AS accepted_order_destination_lat,
  accepted_orders.destination_location_lon AS accepted_order_destination_lon,
  o.pickup_location_lat AS order_in_hand_pickup_lat,
  o.pickup_location_lon AS order_in_hand_pickup_lon,
  o.destination_location_lat AS order_in_hand_destination_lat,
  o.destination_location_lon AS order_in_hand_destination_lon,
  ST_DISTANCE(ST_GEOGPOINT(o.pickup_location_lon, o.pickup_location_lat), ST_GEOGPOINT(o.destination_location_lon, o.destination_location_lat)) as distance_order,
  ST_DISTANCE(ST_GEOGPOINT(driver_lon, driver_lat), ST_GEOGPOINT(o.pickup_location_lon, o.pickup_location_lat)) as distance_from_courier_to_in_hand_pickup,
  ST_DISTANCE(ST_GEOGPOINT(accepted_orders.pickup_location_lon, accepted_orders.pickup_location_lat), ST_GEOGPOINT(accepted_orders.destination_location_lon, accepted_orders.destination_location_lat)) as distance_for_acceptance,
  ST_DISTANCE(ST_GEOGPOINT(driver_lon, driver_lat), ST_GEOGPOINT(accepted_orders.pickup_location_lon, accepted_orders.pickup_location_lat)) as distance_from_courier_to_accepted_pickup,
  ST_DISTANCE(ST_GEOGPOINT(driver_lon, driver_lat), ST_GEOGPOINT(o.destination_location_lon, o.destination_location_lat)) as distance_from_courier_to_in_hand_destination,
  ST_DISTANCE(ST_GEOGPOINT(accepted_orders.pickup_location_lon, accepted_orders.pickup_location_lat), ST_GEOGPOINT(o.destination_location_lon, o.destination_location_lat)) as distance_from_in_hand_destination_to_accepted_pickup,
FROM
  accepted_orders
LEFT JOIN
  last_location
ON
  accepted_orders.courier_id = last_location.courier_id
  AND accepted_orders.id = last_location.order_id
LEFT JOIN
  driver_locat
ON
  driver_locat.driver_id = accepted_orders.courier_id
  AND driver_locat.location_updated_at = last_location.last_location_updated_at
LEFT JOIN
  hand
ON
  accepted_orders.courier_id = hand.courier_id
LEFT JOIN
  completed
ON
  hand.order_id = completed.order_id
LEFT JOIN
  `TableName` o
ON
  hand.order_id = o.system_order_request_id
WHERE
  driver_locat.location_updated_at <= DATETIME(accepted_orders.source_ts,'Asia/Hong_Kong')
  AND hand.event_time<= DATETIME(accepted_orders.source_ts,'Asia/Hong_Kong')
  AND accepted_orders.id <> hand.order_id
  AND completed.event_time > DATETIME(accepted_orders.source_ts,'Asia/Hong_Kong')
  AND o.product_name = 'DELIVERY'
""" 
data = bq_db.df_from_sql(query) 

In [ ]:
df_new = pd.DataFrame(data)
intervals = [-np.inf, 5000, 10000, 15000, 20000, 25000, np.inf]
labels_1 = ['Under 5000 meters', '5000 to 10000 meters', '10000 to 15000', '15000 to 20000', '20000 to 25000', 'More than 25000']
df_new['name_dist'] = pd.cut(np.abs(df_new['distance_order']), intervals, labels=labels_1)
            
value_counts = df_new['name_dist'].value_counts().reset_index()
value_counts.columns = ['Meters', 'Frequency']

fig = px.bar(value_counts, x='Frequency', y='Meters',
             title='Distance between Pick up and Destination')

fig.update_yaxes(categoryorder='array', categoryarray= labels_1)

fig.show()

In [ ]:
intervals_2 = [-np.inf, 500, 1000, 1500, 2000, 2500, 5000, 10000, 20000, np.inf]
labels_2 = ['Under 500 meters', '500 to 1000 meters', '1000 to 1500', '1500 to 2000', '2000 to 2500', '2500 to 5000', '5000 to 10000', '10000 to 20000', 'More than 20000']
df_new['name_courier_to_pickup'] = pd.cut(np.abs(df_new['distance_from_courier_to_in_hand_pickup']), intervals_2, labels=labels_2)
            
value_counts = df_new['name_courier_to_pickup'].value_counts().reset_index()
value_counts.columns = ['Meters', 'Frequency']

fig = px.bar(value_counts, x='Frequency', y='Meters',
             title='Distance between Courier and Hand Pickup')

fig.update_yaxes(categoryorder='array', categoryarray= labels_2)

fig.show()

In [ ]:
intervals_3 = [-np.inf, 2500, 5000, 10000, 15000, 20000, 25000, np.inf]
labels_3 = ['Under 2500 meters', '2500 to 5000', '5000 to 10000 meters', '10000 to 15000', '15000 to 20000', '20000 to 25000', 'More than 25000']
df_new['name_dist_acceptance'] = pd.cut(np.abs(df_new['distance_for_acceptance']), intervals_3, labels=labels_3)
            
value_counts = df_new['name_dist_acceptance'].value_counts().reset_index()
value_counts.columns = ['Meters', 'Frequency']

fig = px.bar(value_counts, x='Frequency', y='Meters',
             title='Distance between Accepted Pick up and Destination')

fig.update_yaxes(categoryorder='array', categoryarray= labels_3)

fig.show()

In [ ]:
df_new['name_courier_to_accepted_pickup'] = pd.cut(np.abs(df_new['distance_from_courier_to_accepted_pickup']), intervals_2, labels=labels_2)
            
value_counts = df_new['name_courier_to_accepted_pickup'].value_counts().reset_index()
value_counts.columns = ['Meters', 'Frequency']

fig = px.bar(value_counts, x='Frequency', y='Meters',
             title='Distance between Courier and Accepted Pickup')

fig.update_yaxes(categoryorder='array', categoryarray= labels_2)

fig.show()

In [ ]:
df_new['name_courier_to_inhand_destination'] = pd.cut(np.abs(df_new['distance_from_courier_to_in_hand_destination']), intervals_2, labels=labels_2)
            
value_counts = df_new['name_courier_to_inhand_destination'].value_counts().reset_index()
value_counts.columns = ['Meters', 'Frequency']

fig = px.bar(value_counts, x='Frequency', y='Meters',
             title='Distance between Courier and In hand Destination')

fig.update_yaxes(categoryorder='array', categoryarray= labels_2)

fig.show()

In [ ]:
df_new['name_inhand_dest_to_accepted_pickup'] = pd.cut(np.abs(df_new['distance_from_in_hand_destination_to_accepted_pickup']), intervals_2, labels=labels_2)
            
value_counts = df_new['name_inhand_dest_to_accepted_pickup'].value_counts().reset_index()
value_counts.columns = ['Meters', 'Frequency']

fig = px.bar(value_counts, x='Frequency', y='Meters',
             title='Distance between In hand Destination and Accepted Pick up')

fig.update_yaxes(categoryorder='array', categoryarray= labels_2)

fig.show()